In [21]:
import numpy
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import RobertaForSequenceClassification
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.amp import autocast, GradScaler
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
#carregando o dataset go_emotions do HuggingFace
dataset_raw = load_dataset('google-research-datasets/go_emotions', 'raw')['train'].shuffle(seed=42)

In [3]:
dataset_val = dataset_raw.select(range(6000))
dataset_treino = dataset_raw.select(range(6000, 176000))

In [ ]:
len(dataset_treino), len(dataset_val)

In [ ]:
dataset_val[0]

In [6]:
emocoes = ['admiration','amusement','anger','annoyance','approval','caring',
    'confusion','curiosity','desire','disappointment','disapproval','disgust',
    'embarrassment','excitement','fear','gratitude','grief','joy','love',
    'nervousness','optimism','pride','realization','relief','remorse',
    'sadness','surprise','neutral']


colunas_necessarias = ['text'] + emocoes

In [7]:
dataset_treino = dataset_treino.remove_columns([c for c in dataset_treino.column_names if c not in colunas_necessarias])
dataset_val = dataset_val.remove_columns([c for c in dataset_val.column_names if c not in colunas_necessarias])

In [ ]:
#instaciando o tokenizer do modelo e criando uma função para tokenizar os dados em batches
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

def tokenizer_batch(batch):
  return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=64)

In [ ]:
#tokenizando os textos dos três conjuntos
dataset_treino = dataset_treino.map(tokenizer_batch, batched=True)
dataset_val = dataset_val.map(tokenizer_batch, batched=True)

In [ ]:
#criando labels no formato que o modelo deve receber
def criar_labels(dados):
    dados['labels'] = [dados[col] for col in emocoes]
    return dados

dataset_treino = dataset_treino.map(criar_labels)
dataset_val    = dataset_val.map(criar_labels)

In [11]:
len(dataset_val[0]['labels'])

28

In [12]:
#passando os dados do dataset para tensores
colunas_modelo = ["input_ids", "attention_mask", "labels"]

dataset_treino.set_format("torch", columns=colunas_modelo)
dataset_val.set_format("torch", columns=colunas_modelo)

In [13]:
#defindo os batches
batch_treino = DataLoader(dataset_treino, batch_size=32, shuffle=True)
batch_validacao = DataLoader(dataset_val, batch_size=32)

In [ ]:
#treinamento do modelo
modelo = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=28, problem_type="multi_label_classification")

In [ ]:
#passando o modelo para rodar na GPU, pois vamos usar uma GPU T4 para o fine-tuning
gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo.to(gpu)

In [ ]:
#adicionando o modelo ao compilador JIT
modelo = torch.compile(modelo)

In [17]:
#instaciando o otimizador
otimizador = AdamW(modelo.parameters(), lr=2e-5, fused=True)

In [ ]:
#treinamento do modelo: 5 epocas com batch size de 32, ou seja, a cada 32 amostras o modelo atualiza os pesos.
epocas = 5
scaler = GradScaler()

for epoca in range(epocas):
    modelo.train()
    total_loss = 0

    for batch in tqdm(batch_treino, desc=f"Época {epoca+1}/{epocas}", leave=False):
        ids = batch["input_ids"].to(gpu)
        mask = batch["attention_mask"].to(gpu)
        labels = batch["labels"].to(gpu).float()

        otimizador.zero_grad()
        with autocast(device_type='cuda'):
            output = modelo(input_ids=ids, attention_mask=mask, labels=labels)
            loss = output.loss

        scaler.scale(loss).backward()
        scaler.step(otimizador)
        scaler.update()

        total_loss += loss.item()

    perda_treino = total_loss / len(batch_treino)
    print(f"Época {epoca+1}/{epocas} | Perda média treino: {perda_treino:.4f}")

    modelo.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in batch_validacao:
            ids = batch["input_ids"].to(gpu)
            mask = batch["attention_mask"].to(gpu)
            labels = batch["labels"].to(gpu).float()

            output = modelo(input_ids=ids, attention_mask=mask, labels=labels)
            val_loss += output.loss.item()

    perda_val = val_loss / len(batch_validacao)
    print(f"Perda média validação: {perda_val:.4f}\n")

In [ ]:
modelo.eval()
todas_pred = []
todas_labels = []

with torch.no_grad():
    for batch in batch_validacao:
        inputs = batch['input_ids'].to(gpu)
        labels = batch['labels'].to(gpu)

        outputs = modelo(inputs)
        preds = torch.sigmoid(outputs.logits)
        preds = (preds > 0.3).int()

        todas_pred.extend(preds.cpu().numpy())
        todas_labels.extend(labels.cpu().numpy())

f1 = f1_score(todas_labels, todas_pred, average='samples')
acuracia = accuracy_score(todas_labels, todas_pred)

print("F1 (multi-label, average='samples'):", f1)
print("Accuracy (multi-label exact match):", acuracia)